# Data Conversion

Sometimes reviews classified as negative by glassdoor are actually neutral.
To identify neutral reviews and create another sentiment class called `neutral`, we will use the pre-trained Hugging Face model [citizenlab/twitter-xlm-roberta-base-sentiment-finetunned](https://huggingface.co/citizenlab/twitter-xlm-roberta-base-sentiment-finetunned) to identify `neutral` reviews and also to ensure that reviews extracted as `positive` and `negative` are truly of their respective class.

In [15]:
import logging
import pandas as pd

from transformers import pipeline

In [16]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler("data_conversion.log"), logging.StreamHandler()],
)

## Loading Model

In [17]:
model_path = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"

In [18]:
sentiment_classifier = pipeline("text-classification", model=model_path)

In [19]:
sentiment_classifier("Essa empresa não tem nada negativo")

[{'label': 'Neutral', 'score': 0.8980973958969116}]

In [20]:
def predict_reviews(
    filtered_reviews_df: pd.DataFrame, expected_label: str
) -> pd.DataFrame:
    predicted_labels = []
    predicted_scores = []

    for index, row in filtered_reviews_df.iterrows():
        review_id = row["review_id"]
        review_text = row["review_text"]

        prediction = sentiment_classifier(review_text)

        predicted_label = prediction[0]["label"]
        predicted_score = prediction[0]["score"]

        if predicted_label != expected_label:
            logging.warning(
                f"Expected label: {expected_label}; Predicted label: {predicted_label}; Score: {predicted_score}; "
                f"review_id: {review_id} review_text: >>{review_text}<< "
            )

        predicted_labels.append(predicted_label)
        predicted_scores.append(predicted_score)

    filtered_reviews_df["predicted_label"] = predicted_labels
    filtered_reviews_df["predicted_score"] = predicted_scores

    return filtered_reviews_df

## Loading reviews

In [21]:
reviews_df = pd.read_csv("./glassdoor_reviews.csv")

In [22]:
reviews_df.head()

,review_id,company,employee_role,employee_detail,review_text,review_date,star_rating,sentiment
0,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano","Companheirismo entre os colegas, oportunidade ...",15 de dez. de 2023,5.0,1
1,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano",Não tive nenhum ponto negativo,15 de dez. de 2023,5.0,-1
2,74420027,Tecnomapas,Analista Desenvolvedor,Ex-freelancer,Equipe bem prestativa e ótima de se trabalhar.,11 de mar. de 2023,4.0,1
3,74420027,Tecnomapas,Analista Desenvolvedor,Ex-freelancer,Modo home office ainda tem que ser melhorado.,11 de mar. de 2023,4.0,-1
4,60212043,Tecnomapas,Funcionário confidencial,"Ex-funcionário(a), menos de um ano",Única vantagem era o trabalho ser home office,24 de fev. de 2022,1.0,1


## Predicting Positive Reviews

In [23]:
positive_reviews_df = reviews_df[reviews_df["sentiment"] == 1]

In [24]:
positive_reviews_df.head()

,review_id,company,employee_role,employee_detail,review_text,review_date,star_rating,sentiment
0,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano","Companheirismo entre os colegas, oportunidade ...",15 de dez. de 2023,5.0,1
2,74420027,Tecnomapas,Analista Desenvolvedor,Ex-freelancer,Equipe bem prestativa e ótima de se trabalhar.,11 de mar. de 2023,4.0,1
4,60212043,Tecnomapas,Funcionário confidencial,"Ex-funcionário(a), menos de um ano",Única vantagem era o trabalho ser home office,24 de fev. de 2022,1.0,1
6,60144095,Tecnomapas,Analista,Funcionário(a) atual,"Ambiente colaborativo, equipe preparada e como...",22 de fev. de 2022,5.0,1
8,61592206,Tecnomapas,Técnico Em Geoprocessamento,"Ex-funcionário(a), mais de 3 anos",Empresa que ensina muito e contrata sem experi...,22 de mar. de 2022,1.0,1


In [25]:
positive_reviews_df.shape

(1283, 8)

In [ ]:
predicted_positive_reviews = predict_reviews(positive_reviews_df, "Positive")

In [12]:
predicted_positive_reviews.head()

,review_id,company,employee_role,employee_detail,review_text,review_date,star_rating,sentiment,predicted_label,predicted_score
0,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano","Companheirismo entre os colegas, oportunidade ...",15 de dez. de 2023,5.0,1,Neutral,0.814863
2,74420027,Tecnomapas,Analista Desenvolvedor,Ex-freelancer,Equipe bem prestativa e ótima de se trabalhar.,11 de mar. de 2023,4.0,1,Positive,0.987299
4,60212043,Tecnomapas,Funcionário confidencial,"Ex-funcionário(a), menos de um ano",Única vantagem era o trabalho ser home office,24 de fev. de 2022,1.0,1,Neutral,0.980318
6,60144095,Tecnomapas,Analista,Funcionário(a) atual,"Ambiente colaborativo, equipe preparada e como...",22 de fev. de 2022,5.0,1,Positive,0.949690
8,61592206,Tecnomapas,Técnico Em Geoprocessamento,"Ex-funcionário(a), mais de 3 anos",Empresa que ensina muito e contrata sem experi...,22 de mar. de 2022,1.0,1,Negative,0.960868


In [13]:
predicted_non_positive_reviews = predicted_positive_reviews[
    predicted_positive_reviews["predicted_label"] != "Positive"
]

In [14]:
predicted_non_positive_reviews.shape

(650, 10)

In [15]:
predicted_non_positive_reviews.to_csv("predicted_non_positive_reviews.csv", index=False)

## Predicting Negative Reviews

In [26]:
negative_reviews_df = reviews_df[reviews_df["sentiment"] == -1]

In [27]:
negative_reviews_df.head()

,review_id,company,employee_role,employee_detail,review_text,review_date,star_rating,sentiment
1,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano",Não tive nenhum ponto negativo,15 de dez. de 2023,5.0,-1
3,74420027,Tecnomapas,Analista Desenvolvedor,Ex-freelancer,Modo home office ainda tem que ser melhorado.,11 de mar. de 2023,4.0,-1
5,60212043,Tecnomapas,Funcionário confidencial,"Ex-funcionário(a), menos de um ano",Empresa não oferece salário compatível com mer...,24 de fev. de 2022,1.0,-1
7,60144095,Tecnomapas,Analista,Funcionário(a) atual,Nao tem contras nessa empresa,22 de fev. de 2022,5.0,-1
9,61592206,Tecnomapas,Técnico Em Geoprocessamento,"Ex-funcionário(a), mais de 3 anos","Salários baixos, sem benéficos e ambiente hostil",22 de mar. de 2022,1.0,-1


In [28]:
negative_reviews_df.shape

(1283, 8)

In [ ]:
predicted_negative_reviews = predict_reviews(negative_reviews_df, "Negative")

In [11]:
predicted_negative_reviews.head()

,review_id,company,employee_role,employee_detail,review_text,review_date,star_rating,sentiment,predicted_label,predicted_score
1,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano",Não tive nenhum ponto negativo,15 de dez. de 2023,5.0,-1,Neutral,0.879178
3,74420027,Tecnomapas,Analista Desenvolvedor,Ex-freelancer,Modo home office ainda tem que ser melhorado.,11 de mar. de 2023,4.0,-1,Neutral,0.932563
5,60212043,Tecnomapas,Funcionário confidencial,"Ex-funcionário(a), menos de um ano",Empresa não oferece salário compatível com mer...,24 de fev. de 2022,1.0,-1,Negative,0.860661
7,60144095,Tecnomapas,Analista,Funcionário(a) atual,Nao tem contras nessa empresa,22 de fev. de 2022,5.0,-1,Neutral,0.978598
9,61592206,Tecnomapas,Técnico Em Geoprocessamento,"Ex-funcionário(a), mais de 3 anos","Salários baixos, sem benéficos e ambiente hostil",22 de mar. de 2022,1.0,-1,Negative,0.899390


In [12]:
predicted_non_negative_reviews = predicted_negative_reviews[
    predicted_negative_reviews["predicted_label"] != "Negative"
]

In [13]:
predicted_non_negative_reviews.shape

(821, 10)

In [14]:
predicted_non_negative_reviews.to_csv("predicted_non_negative_reviews.csv", index=False)